## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## Loading all the enviroment varaible

groq_api_key = os.getenv("GROQ_API_KEY")


In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f8351bf5400>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f8351bf7770>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Sadeeq and I and a Chief AI Engineer")])

AIMessage(content="Nice to meet you, Sadeeq! It's great to hear from a fellow AI enthusiast. \n\nAs a Chief AI Engineer, I imagine you're involved in some fascinating projects. What kind of work are you currently focused on? \n\nI'm always eager to learn more about the cutting edge of AI development. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 73, 'prompt_tokens': 23, 'total_tokens': 96, 'completion_time': 0.132727273, 'prompt_time': 0.002189854, 'queue_time': 0.241706515, 'total_time': 0.134917127}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1dfc01a4-f98b-409b-8a33-5680264b1c33-0', usage_metadata={'input_tokens': 23, 'output_tokens': 73, 'total_tokens': 96})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Sadeeq and I and a Chief AI Engineer"),
        AIMessage(content="Hi Sadeeq, it's nice to meet you! That's a very interesting role. \n\nWhat kind of projects are you working on as a Chief AI Engineer? I'm always curious to learn more about the applications of AI.\n"),
        HumanMessage(content="Hey Whats my name and what do i do?")
    ]
)

AIMessage(content="You are Sadeeq, and you are a Chief AI Engineer. \n\nIs there anything else you'd like to tell me about yourself or your work? I'm eager to learn more! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 94, 'total_tokens': 139, 'completion_time': 0.081818182, 'prompt_time': 0.005982004, 'queue_time': 0.245486346, 'total_time': 0.087800186}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f2371d8d-4967-4444-b6d7-c5bb7d39bae7-0', usage_metadata={'input_tokens': 94, 'output_tokens': 45, 'total_tokens': 139})

### Message History
we can use a Message history class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

## Creating Session for each user using the model
store={}
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [6]:
config = {"configurable":{"session_id":"chat1"}}

In [7]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Sadeeq and I and a Chief AI Engineer")],
    config=config
)

In [8]:
response.content

"Hi Sadeeq, it's nice to meet you!\n\nThat's a fascinating title. As a Chief AI Engineer, I imagine you're at the forefront of some exciting developments. What kind of projects are you working on?  \n\nI'm eager to learn more about your work in the field of AI.\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config
)

AIMessage(content='Your name is Sadeeq.  \n\nI remember it from our first exchange! 😄  How can I help you today, Sadeeq? \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 103, 'total_tokens': 135, 'completion_time': 0.058181818, 'prompt_time': 0.005084376, 'queue_time': 0.243241753, 'total_time': 0.063266194}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a1f6eb5b-fb53-4db0-9769-c10edb7c7041-0', usage_metadata={'input_tokens': 103, 'output_tokens': 32, 'total_tokens': 135})

In [10]:
### Change the config (session iD) to see whether the model will remember

config1 = {"configurable":{"session_id":"chat2"}}
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content


"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to know!\n"

In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey, My name is Muhammad Sadeeq")],
    config=config1
)
response.content


"Hi Muhammad Sadeeq, it's nice to meet you! \n\nIs there anything I can help you with today? 😊  \n\n"

In [12]:
response = with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is Muhammad Sadeeq.  \n\nI remembered!  😊  How can I help you further?\n'

### Prompt Template

Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. let's now make that bit more complicated.
First, let's add in a system message with some custom instruction (but still taking messages as input). Next we'll add in more input besides jusr the messages

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant. Answer all question to the best of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain  = prompt | model 

In [20]:
chain.invoke({"messages": [HumanMessage(content="Hi My name is Sadeeq")]})

AIMessage(content="Hi Sadeeq! It's nice to meet you.  \n\nWhat can I do for you today? 😊  I'm ready to answer your questions and help in any way I can. \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 45, 'prompt_tokens': 30, 'total_tokens': 75, 'completion_time': 0.081818182, 'prompt_time': 0.002322175, 'queue_time': 0.296659834, 'total_time': 0.084140357}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3ef5b5a4-5c29-466f-9f34-add9f1fa404c-0', usage_metadata={'input_tokens': 30, 'output_tokens': 45, 'total_tokens': 75})

In [22]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)


In [23]:
config = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [HumanMessage(content="Hi My name is Sadeeq")],
    config=config
)
response.content

"Hello Sadeeq! It's nice to meet you. \n\nWhat can I do for you today? 😊  \n\n"

In [24]:
### Adding More Complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            "You are a helpful assistant. Answer all question to the best of your ability in {language}"
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain  = prompt | model 

In [27]:
response = chain.invoke({"messages": [HumanMessage(content="Hi My name is Sadeeq")], "language": "french"})
response.content

"Bonjour Sadeeq ! 👋 \n\nEnchanté de te rencontrer. Comment puis-je t'aider aujourd'hui ? 😊 \n"

lets now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history

In [28]:
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

In [29]:
config = {"configurable":{"session_id":"chat4"}}
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi, I am Sadeeq")], "language":"Hausa"},
    config=config
)
response.content

"Sannu da zuwa Sadeeq!  😊  Ina cikin aiki don taimakawa.\n\nShikenan, ka tambayi abin da zaka yi. Wasu tambayoyin da nake iya amsawa sune:\n\n* Yana yawa hausa\n* Tambayoyi game da al'umma\n* Gudanar da batutuka\n* Sarrafa bayanai\n* Kuma yawa da yawa.\n\nKamar yadda kike bukata, Sadeeq!  \n"

In [30]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hey, whats my name")], "language":"Hausa"},
    config=config
)
response.content

'Anan take sunan ka, Sadeeq! 😊 \n'

In [31]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hey, Tell me a short story")], "language":"Hausa"},
    config=config
)
response.content

'A tsakiyar yankin Kano, akwai matashi mai suna Fatima, mai kula da shanu. Tana da sha\'awar yaran da ta kula da su kamar yaranta.\n\nA rana mai zafi, Fatima ta samu yarinya mai suna Hadiza, tana kuka a kofar gidanta. Hadiza ta ce ta rasa hanyar gida bayan ta bi kansa mai sauri. Fatima ta ji tausayin Hadiza, ta karbi ta, ta saita ta da ruwan zamani, ta bata abinci, ta kuma ji tsohon almara.\n\nSai dai, lokacin da Fatima ta fara bin hanyar da Hadiza ta ce ta zo daga, ta gano cewa ba ta san inda take ba. Fatima ta rasa abinda zata yi, sai dai ta yi tambayar mutanen da suka fuskanta. \n\nA karshe, mutane sun taimaka mata, suka nuna mata hanyar da Hadiza ta zo daga. Fatima ta yi farin ciki sosai, ta juya Hadiza ta gida, ta kuma yi alkawarin cewa za ta kula da ita kamar yarinta.\n\nHadiza ta yi murmushi a fuskarta, ta goya Fatima da goge, ta ce "Na gode sosai, Fatima." Fatima ta ce "A\'a. Ina jin farin ciki ne saboda na iya taimakawa."\n\nA haka ne Fatima ta yi martabar iyawon mata, ta kuma

### Manage the conservation History

One important concept of understanding when building chatbot is how to manage conversation history. If left unmanaged, the messages will grow unbounded and potentially overfloe the content window of the LLM. Therefore, it is important to add a step that limits the size of the messages yor are passin in.

"trim_ messages" help to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [38]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens = 45, ## Change to 45 from 70
    strategy="last",
    token_counter = model,
    include_system=True,
    allow_partial=False,
    start_on = "human"
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="Hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="i like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="Whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="No problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes1")
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='i like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='No problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes1', additional_kwargs={}, response_metadata={})]

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what ice cream do i like")],
        "language":"English"
    }
)
response.content

### output = I don't have access to your personal information
### Bcos the max_token is set to 45 and and ice is not in the range

"As a large language model, I don't have access to your personal information, including your taste preferences. \n\nWhat's your favorite flavor of ice cream?\n"

In [40]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language":"English"
    }
)
response.content

'You asked "What\'s 2 + 2?".  😊 \n'

In [47]:
## Lets wrap this in the message histoy
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config = {"configurable":{"session_id":"chat5"}}

In [48]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hey, whats my name")], 
     "language":"English"},
    config=config
)
response.content

"As an AI, I don't have memory of past conversations. So I don't know your name.  What would you like me to call you? 😊  \n"

In [50]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask")],
        "language":"English"
    },
    config=config
)
response.content

'Please provide me with the math problem you asked so I can help you! I have no memory of past conversations, so I need you to tell me the problem again. 😊  \n\n'